# Getting Started Notebook

Hello! 

This notebooks should be where you start for a walkthrough on how to use this code package. It is not super clean, so please send notes if you find it confusing.

This starts with a lot of imports because I haven't quite had the time to clean the code in a way that prunes these down.

In [ ]:
%load_ext autoreload
%autoreload 2
# This is so that editing the config files for the tokamaks
# updates the config in a notebook style.

# Move into the source directory for this notebook to work properly
# Probably want a better way of doing this.
import os
os.chdir('../src/')

# Basic Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the core disruptivity functions.
import disruptivity as dis

# Import visualizers.
import vis.disruptivity_vis as dis_vis
from vis.plot_helpers import plot_subplot as plot

# Impor the mat file data loader.
from data_loader import load_disruptions_mat as load_disruptions_mat

## CMOD
### File Loading

Now that the configuration is loaded, let's import some CMOD data. This is in a mat file, but after loading it in, it is converted into a pandas dataframe `cmod_df` and a dictionary of indices organized by label, `cmod_indices`.

In the dataframe, collumns are indexed by the name of the parameter. All the pulses in the database are organized in a flattened line.

In the indices dictionary, indices are sorted in their chronological order and label. Let's print out the labels and see how it looks.

*NOTE: The time resolution is non uniform. This is important for later.*

In [ ]:
cmod_df, cmod_indices = load_disruptions_mat('../data/CMod_disruption_warning_db.mat')
print(cmod_indices.keys())

Notice that these keys might have confusing names. The important ones are 
- `indices_disrupt`: Indices belonging to a disrupted pulse.
- `indices_flattop`: Indices belonging to the flattop of a pulse.
- `indices_disrupt_in_flattop`: Indices for pulses that disrupt in flattop (??)
- `indices_disrupt_time`: The time of disruption indices.

This demo starts by introducing how Tokamaks are configured in this codebase. They are represented by simple dictionaries in the `./src/tokamaks` folder. They include some metadata on the tokamak, as well as entries that describe the regions of interest for parameters we will use for analysis. The keys of the entries must match the headers of the columns in the dataframe imported above.

The dictionary structure is pretty self descriptive, so please open a file and just read through the keys.

In [ ]:
from tokamaks.cmod import CONFIG as CMOD

## Using Tokamak Information

We will use the above dictionary to build the axes over which we will compute disruptivity. The axes are described by entry dictionaries or the conventionally called `entry_dict`. The entry dict is keyed into by the name of the parameter that one wishes to compute the disruptivity over. These provide other parameters like the range of the histogram that will be used for disruptivity calculations and the axis name for plotting.

The number of entries in the entry dictionary set the dimension of the disruptivity calculation.

Let us configure a entry dictionary that will compute the disruptivity over 1D $\kappa$.

In [ ]:
# Manually creating an entry dict
entry_dict = {
    "kappa": {
            "range": [0.8, 2.0],
            "axis_name": "$\kappa$",
    },
}

# But, don't do that, instead use the tokamak configuration to 
# build these in a more reliable way.
entry_dict = {
    "kappa": CMOD['entry_dict']['kappa']
}

Now we need to compute the disruptivity itself. For the sampling method, this is divided into two steps.
1. Define the disruption and histogram the data.
2. Compute the disruptivy map from those histograms.

The reason for this separation is that there are further things that can be set in the disruptivity calculation. There are a few settings but the most notable ones at this stage are the $\tau$ time and the window size (in ms measured backwards from the tau time) for the label of the disrupted data point. Generally we want the window to be the same size as `dt` near the tau time.

*Note: Currently the code base only supports computing the disruptivity during the flattop. Extending this is easy and if we need it just ask.*

In [ ]:
# Compute indices of interest. Usually this is the numerator and denominator in the disruptivity calculations.
# Here we do tau = 0 ms and window = 2 ms
indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(CMOD, cmod_df, cmod_indices, tau=0, window=2)
                                                                  
# Next comput the disruptivity itself from the indices.
# Here we use the sampling method instead of the ML method.
args = dis.compute_disruptivity_sampling(cmod_df, entry_dict, indices_n_disrupt, indices_n_total, nbins=35)

# Finally plot the output.
# TODO: I will need to document how plotting works in this better
fig,ax = plot("cmod_kappa_zerr_disruptivity.png", dis_vis.subplot_disruptivity1d, args)

In [ ]:
# Finally as a little cherry on top, let's try a 2D calculation.
entry_dict_2D = {
    'kappa': CMOD['entry_dict']['kappa'],
    'beta_n': CMOD['entry_dict']['beta_n'],
}

# No need to recompute the histogram. It is good to go.
args = dis.compute_disruptivity_sampling(cmod_df, entry_dict_2D, indices_n_disrupt, indices_n_total, nbins=35)

# Finally plot the output. Switch to the 2D plotter.
fig,ax = plot("cmod_kappa_zerr_disruptivity.png", dis_vis.subplot_disruptivity2d, args)

Yay! That should be just enough to jump start you. If you wanna try DIII-D out. Just do everything here but with the DIII-D data and structures and it will work.